In [30]:
# Imports
import requests
import matplotlib.pyplot as plt
import numpy as np 
from pathlib import Path 
import json
import pandas as pd
from pathlib import Path
import os
from dotenv import load_dotenv
import csv
from bs4 import BeautifulSoup
from sklearn import linear_model 
from sklearn import svm
from sklearn import ensemble
import warnings 

In [31]:
warnings.filterwarnings('ignore')

In [2]:
#read_csv file for "FF_qb_2020.csv" and "FF_qb_2021.csv"
qb_data_2020 = pd.read_csv("FF_qb_2020.csv")
qb_data_2021 = pd.read_csv("FF_qb_2021.csv")
qb_data_2020

,Rank,Player,CMP,ATT,PCT,YDS,Y/A,TD,INT,SACKS,ATT.1,YDS.1,TD.1,FL,G,FPTS,FPTS/G,ROST
0,1,Josh Allen (BUF),396,572,69.2,"4,544",7.9,37,10,26,102,421,8,6,16,405.2,25.3,100.00%
1,2,Kyler Murray (ARI),375,558,67.2,"3,971",7.1,26,12,27,133,819,11,4,16,390.7,24.4,98.50%
2,3,Aaron Rodgers (GB),372,526,70.7,"4,299",8.2,48,5,20,38,149,3,2,16,387.1,24.2,96.00%
3,4,Patrick Mahomes II (KC),390,588,66.3,"4,740",8.1,38,6,22,62,308,2,2,15,380.3,25.4,100.00%
4,5,Deshaun Watson (CLE),382,544,70.2,"4,823",8.9,33,7,49,90,444,3,3,16,376.4,23.5,15.80%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,81,Easton Stick (LAC),1,1,100.0,4,4.0,0,0,0,1,-2,0,0,1,0.0,0.0,0.00%
81,82,Mike White (NYJ),0,0,0.0,0,0.0,0,0,0,0,0,0,0,1,0.0,0.0,0.00%
82,83,Matt Schaub (FA),0,0,0.0,0,0.0,0,0,0,3,-4,0,0,1,-0.4,-0.4,0.00%
83,84,Nate Sudfeld (DET),5,12,41.7,32,2.7,0,1,2,2,12,0,1,1,-0.5,-0.5,0.00%


In [3]:
#Get column data for Player, Fantasy Points, and Fantasay points a game. dropna. 
qb_data_2020 = qb_data_2020[["Player" , "FPTS", "FPTS/G"]].dropna()
qb_data_2021 = qb_data_2021[["Player" , "FPTS", "FPTS/G"]].dropna()

In [4]:
#Add years and QB's to the table names using add_prefix function
qb_data_2020 = qb_data_2020.add_prefix("2020_QBs_")
qb_data_2021 = qb_data_2021.add_prefix("2021_QBs_")

In [5]:
#Display data
qb_data_2020

,2020_QBs_Player,2020_QBs_FPTS,2020_QBs_FPTS/G
0,Josh Allen (BUF),405.2,25.3
1,Kyler Murray (ARI),390.7,24.4
2,Aaron Rodgers (GB),387.1,24.2
3,Patrick Mahomes II (KC),380.3,25.4
4,Deshaun Watson (CLE),376.4,23.5
...,...,...,...
80,Easton Stick (LAC),0.0,0.0
81,Mike White (NYJ),0.0,0.0
82,Matt Schaub (FA),-0.4,-0.4
83,Nate Sudfeld (DET),-0.5,-0.5


In [6]:
#Dispaly data
qb_data_2021

,2021_QBs_Player,2021_QBs_FPTS,2021_QBs_FPTS/G
0,Josh Allen (BUF),417.7,24.6
1,Justin Herbert (LAC),395.6,23.3
2,Tom Brady (TB),386.7,22.7
3,Patrick Mahomes II (KC),374.2,22.0
4,Matthew Stafford (LAR),346.8,20.4
...,...,...,...
117,Davis Webb (NYG),-0.3,-0.3
118,Feleipe Franks (ATL),-0.4,-0.1
119,Logan Woodside (TEN),-0.6,-0.2
120,John Wolford (LAR),-0.9,-0.5


In [10]:
# Merge datasets using SQL joins: how="outer", left_on, right_on and dropna.
merged_qb_data= pd.merge(qb_data_2020, qb_data_2021, how="outer", left_on= "2020_QBs_Player", right_on="2021_QBs_Player").dropna()

# I want to compare two files and make sure they're in the exact same order

In [26]:
#Merge datasets and make sure they're in the exact order and dropna.
combined_matched_index= pd.merge(qb_data_2020, qb_data_2021, left_index =True, right_index=True, how='outer').dropna()

In [27]:
combined_matched_index

,2020_QBs_Player,2020_QBs_FPTS,2020_QBs_FPTS/G,2021_QBs_Player,2021_QBs_FPTS,2021_QBs_FPTS/G
0,Josh Allen (BUF),405.2,25.3,Josh Allen (BUF),417.7,24.6
1,Kyler Murray (ARI),390.7,24.4,Justin Herbert (LAC),395.6,23.3
2,Aaron Rodgers (GB),387.1,24.2,Tom Brady (TB),386.7,22.7
3,Patrick Mahomes II (KC),380.3,25.4,Patrick Mahomes II (KC),374.2,22.0
4,Deshaun Watson (CLE),376.4,23.5,Matthew Stafford (LAR),346.8,20.4
...,...,...,...,...,...,...
80,Easton Stick (LAC),0.0,0.0,Brett Ratliff (TEN),0.0,0.0
81,Mike White (NYJ),0.0,0.0,Brock Purdy (SF),0.0,0.0
82,Matt Schaub (FA),-0.4,-0.4,Kyle Trask (TB),0.0,0.0
83,Nate Sudfeld (DET),-0.5,-0.5,Skylar Thompson (MIA),0.0,0.0
